### Setup

In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
climate_train = pd.read_csv("./drive/MyDrive/Colaboratory Notebooks/WiDS22_Climate_Change/Data/train.csv")
climate_train = climate_train.set_index("id")
climate_train.sample(3)

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32337,3,State_6,Residential,Multifamily_Uncategorized,51138.0,1930.0,NaN,42.7,11,35.080645,61,17,33.910714,55,27,40.145161,59,33,52.983333,82,42,62.790323,90,53,72.650000,92,64,79.838710,98,59,74.596774,90,47,67.866667,96,40,60.193548,86,23,45.250000,70,19,38.548387,71,1272,4670,46.32,29.6,64,55.449315,50,11,0,0,75,11,0,0,1.0,1.0,1.0,99.0,77.979178
38370,4,State_6,Commercial,Office_Uncategorized,528180.0,1981.0,27.0,42.7,4,28.596774,58,9,31.625000,56,13,37.661290,66,31,52.333333,77,49,64.000000,86,55,72.450000,89,63,76.145161,91,61,74.516129,90,52,69.733333,92,42,59.629032,77,22,45.300000,69,24,40.516129,65,1128,4875,53.79,50.0,377,54.501370,66,24,7,0,75,4,0,0,1.0,1.0,1.0,NaN,144.619629
51075,5,State_6,Residential,Multifamily_Uncategorized,57000.0,1910.0,14.0,21.3,8,29.854839,56,2,23.946429,43,12,38.096774,62,32,54.250000,80,48,68.516129,88,50,71.183333,90,64,78.790323,96,63,78.967742,95,56,74.450000,97,35,58.048387,78,32,52.783333,74,34,50.774194,72,1581,4460,40.97,49.1,463,56.835616,63,34,9,0,106,12,0,0,NaN,NaN,NaN,NaN,120.880115


## Data cleaning and preprocessing

Energy Star and Facility Type

### Missing and extreme values

In [ ]:
climate_copy = climate_train.copy(deep = True)
climate_copy.sample(3)

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
45275,5,State_6,Residential,Multifamily_Uncategorized,100879.0,1985.0,NaN,15.2,8,29.854839,56,2,23.946429,43,12,38.096774,62,32,54.250000,80,48,68.516129,88,50,71.183333,90,64,78.790323,96,63,78.967742,95,56,74.450000,97,35,58.048387,78,32,52.783333,74,34,50.774194,72,1581,4460,40.97,49.1,463,56.835616,63,34,9,0,106,12,0,0,NaN,NaN,NaN,NaN,80.924531
48351,5,State_6,Commercial,Office_Uncategorized,65000.0,1918.0,22.0,21.3,8,29.854839,56,2,23.946429,43,12,38.096774,62,32,54.250000,80,48,68.516129,88,50,71.183333,90,64,78.790323,96,63,78.967742,95,56,74.450000,97,35,58.048387,78,32,52.783333,74,34,50.774194,72,1581,4460,40.97,49.1,463,56.835616,63,34,9,0,106,12,0,0,NaN,NaN,NaN,NaN,78.090945
27983,3,State_6,Commercial,Office_Uncategorized,135000.0,1896.0,83.0,15.2,11,35.080645,61,17,33.910714,55,27,40.145161,59,33,52.983333,82,42,62.790323,90,53,72.650000,92,64,79.838710,98,59,74.596774,90,47,67.866667,96,40,60.193548,86,23,45.250000,70,19,38.548387,71,1272,4670,46.32,29.6,64,55.449315,50,11,0,0,75,11,0,0,NaN,NaN,NaN,NaN,147.348108


Drop features with significant amount of missing values

In [ ]:
# Drop features with siginificant amount of missing values
climate_clean = climate_copy.drop(columns=['direction_max_wind_speed', 'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog'])

Drop rows where year_built is 0

In [ ]:
# year_built 0 rows - drop (treat as NaN)
len(climate_clean.loc[climate_clean['year_built'] == 0])

6

In [ ]:
climate_clean = climate_clean[climate_clean.year_built != 0]
# climate_clean[climate_clean.year_built == 0]

### TODO: impute energy_star_rating missing values
Formula or predict values with ML - trade off of less data vs prediction on top of prediction

Baseline: dropped rows with missing values

2nd Attempt: impute with mean

In [ ]:
# Rows in dataset - 75k
print(len(climate_clean.index))

75751


In [ ]:
#  Missing values in energy_star_rating - 26k
climate_clean['energy_star_rating'].isna().sum()

26709

In [ ]:
#  Missing values in year_built - 2k
climate_clean['year_built'].isna().sum()

1837

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(climate_clean[['energy_star_rating']])
climate_clean['energy_star_rating'] = imputer.transform(climate_clean[['energy_star_rating']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# Drop missing values for year_built (TODO)
climate_clean = climate_clean.dropna()

In [ ]:
#climate_clean.isna().sum()

In [ ]:
print(len(climate_clean.index))

73914


### Categorical -> numerical

Baseline: one hot encode facility_type

2nd attempt: label encode

3rd attempt: cluster into fewer groups

Categorical columns: State_Factor, building_class, facility_type

In [ ]:
climate_clean['facility_type'].nunique()

60

In [ ]:
# Categorical columns: 'State_Factor', 'building_class', 'facility_type'
climate_clean.State_Factor.describe()

count       73914
unique          7
top       State_6
freq        50007
Name: State_Factor, dtype: object

In [ ]:
climate_clean.State_Factor.sample(3)

id
68744    State_8
25038    State_6
43180    State_6
Name: State_Factor, dtype: object

In [ ]:
climate_clean.building_class.describe()

count           73914
unique              2
top       Residential
freq            43150
Name: building_class, dtype: object

In [ ]:
climate_clean.building_class.sample(3)

id
6175     Residential
57555    Residential
68341    Residential
Name: building_class, dtype: object

In [ ]:
climate_clean.facility_type.describe()

count                         73914
unique                           60
top       Multifamily_Uncategorized
freq                          39072
Name: facility_type, dtype: object

In [ ]:
climate_clean.facility_type.sample(3)

id
31906             Multifamily_Uncategorized
2671     Mixed_Use_Predominantly_Commercial
64681             Multifamily_Uncategorized
Name: facility_type, dtype: object

In [ ]:
climate_clean.cooling_degree_days.describe()

count    73914.000000
mean      1201.973402
std        385.793403
min         55.000000
25%       1128.000000
50%       1277.000000
75%       1489.000000
max       1999.000000
Name: cooling_degree_days, dtype: float64

* State_Factor -> dummy
*building_class -> dummy
* facility_type -> dummy (for now) -> label encode later
* Year_Factor -> dummy (assumption: order not kept)

In [ ]:
# Make 4 nominal categorical variables into dummy variables (one-hot encoding)
climate_n = pd.get_dummies(climate_clean, columns = ['State_Factor', 'building_class', 'facility_type', 'Year_Factor'])

In [ ]:
# Label encode facility_type
# from sklearn.preprocessing import LabelEncoder
# labelencoder = LabelEncoder()
# climate_n['facility_type'] = labelencoder.fit_transform(climate_n['facility_type'])
# climate_n['facility_type'].sample(3)

In [ ]:
print(climate_n.shape) # Expect (63 - 4) + (41 + 2 + 7 + 6 - 4) = 111 ?

(73914, 130)


In [ ]:
print(list(climate_n.columns))

['floor_area', 'year_built', 'energy_star_rating', 'ELEVATION', 'january_min_temp', 'january_avg_temp', 'january_max_temp', 'february_min_temp', 'february_avg_temp', 'february_max_temp', 'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp', 'august_max_temp', 'september_min_temp', 'september_avg_temp', 'september_max_temp', 'october_min_temp', 'october_avg_temp', 'october_max_temp', 'november_min_temp', 'november_avg_temp', 'november_max_temp', 'december_min_temp', 'december_avg_temp', 'december_max_temp', 'cooling_degree_days', 'heating_degree_days', 'precipitation_inches', 'snowfall_inches', 'snowdepth_inches', 'avg_temp', 'days_below_30F', 'days_below_20F', 'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F', 'days_above_100F',

In [ ]:
# TODO: label encode facility_type (think of numeric encoding that keeps groupings and makes sense as numbers distance-wise)
# Make groupings with clustering for fewer unique types

### Output

In [ ]:
# Export clean + preprocessed csv (climate_n)
climate_n.to_csv('./train_clean_2.csv')